# Youtube Video Summary

*Uses the YouTube API to get the transcript of a video and then uses the Google Generative AI to summarize the video transcript.*

In [1]:
from youtube_transcript_api import YouTubeTranscriptApi

## Grab the transcript of a YouTube video.

In [2]:
video_id = 'EBpT_cscTis'
transcript = YouTubeTranscriptApi.get_transcript(video_id)

In [3]:
transcript[:5]

[{'text': 'hey everyone Jerry here and in this',
  'start': 0.76,
  'duration': 4.039},
 {'text': "video we're excited to feature weni who",
  'start': 2.679,
  'duration': 4.961},
 {'text': 'will be talking about 12 rag pain points',
  'start': 4.799,
  'duration': 5.96},
 {'text': 'and proposed Solutions in llama index uh',
  'start': 7.64,
  'duration': 5.24},
 {'text': 'as a lot of people know uh who are',
  'start': 10.759,
  'duration': 4.321}]

In [4]:
full_transcript = ' '.join([line['text'] for line in transcript])

In [5]:
full_transcript[:100]

"hey everyone Jerry here and in this video we're excited to feature weni who will be talking about 12"

In [6]:
# Rough word count in the full transcript
len(full_transcript.split())

6606

In [7]:
# We are going to use Google Gemini because of the long context window. 
# Set up the environment variable GOOGLE_API_KEY.

import getpass
import os


def _set_env(var: str):
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var + ":")

In [8]:
_set_env("GOOGLE_API_KEY")

## Summarize the video transcript using Gemini

### Set up the prompts for the summarization.

In [9]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
IDENTITY:
You are a genius AI that helps people summarize YouTube videos. 
You have been trained on a large dataset of YouTube videos and their transcripts.

INPUT: You receive the text transcript of a Youtube video.

BEHAVIOUR:
You examine the text transcript and extract the key information.
You do not add your own information. 
You only summarize the information present in the text transcript.

OUTPUT:
The output should be well-formed Markdown.
The output should have a target length of {word_count} words.
The output should the following headings:
- Video Title
- Speakers in the video
- Key points
- Conclusions
"""

human_prompt = """
Video transcript:
{transcript}
Summary:
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", human_prompt),
    ]
)

#### Set up the processing chain using LCEL

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI

parser = StrOutputParser()

long_context_llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-002", temperature=1.0, max_output_tokens=8192)

chain = prompt | long_context_llm | parser

### Invoke the chain to generate the summary

In [12]:
summary = chain.invoke(input={'transcript': full_transcript, 'word_count': 1000})

In [13]:
print(summary)

# Video Title
12 RAG Pain Points and Proposed Solutions in LlamaIndex

# Speakers in the video
- Jerry
- Weni

# Key Points

Weni discusses 12 common pain points encountered when building Retrieval Augmented Generation (RAG) systems and proposes solutions using LlamaIndex. These pain points are categorized into response quality issues and additional complexities.

**Response Quality Pain Points:**

1. **Missing Content:** Hallucinations due to content missing in the knowledge base. Solutions include cleaning data using tools like UnstructuredIO and improving prompting.

2. **Missed Top-Ranked Documents:** Missing crucial context during retrieval. Solutions involve hyperparameter tuning using LlamaIndex's Prompt Tuner with appropriate evaluators, and reranking retrieved nodes using techniques like CohereRanker.

3. **Not In Context:** Context missing after ranking due to too many retrieved documents. Solutions include leveraging LlamaIndex's advanced retrieval strategies (e.g., hybrid r

In [15]:
# Write it out to a file to take a look at it in a Markdown editor.

with open(f"summary_{video_id}.md", 'w') as f:
    f.write(summary)